# Convolutional Neural Network (CNN) Model

- BOOK: [Predict the Future with MLPs, CNNs and LSTMs in Python](https://machinelearningmastery.com/deep-learning-for-time-series-forecasting/) by Jason Brownlee

In [ ]:
# !pip3 uninstall -y torch torchvision
# !pip3 install torch torchvision

In [1]:
import os
import sys
import torch

import numpy as np
import pandas as pd

import torch.nn as nn
import torch.optim as optim

# Get the current working directory of the notebook
notebook_dir = os.getcwd()

# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../framework_for_time_series_data/tslearn/'))
from ml_models import MLP
from ts_models import EvaluationMetric

In [2]:
observations = [10, 20, 30, 40, 50, 60, 70, 80, 90]

# Book's implementation

In [3]:
def split_sequence(sequence, n_steps_in, n_steps_out): 
    X, y = list(), list() 
    for i in range(len(sequence)): 
        # find the end of this pattern 
        end_ix = i + n_steps_in 
        out_end_ix = end_ix + n_steps_out 
        # check if we are beyond the sequence 
        if out_end_ix > len(sequence): 
            break
        # gather input and output parts of the pattern 
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix] 
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [4]:
X, y = split_sequence(observations, 3, 1)

In [5]:
X

array([[10, 20, 30],
       [20, 30, 40],
       [30, 40, 50],
       [40, 50, 60],
       [50, 60, 70],
       [60, 70, 80]])

In [6]:
y

array([[40],
       [50],
       [60],
       [70],
       [80],
       [90]])

# My implementation

In [ ]:
def convert_uts_sequence_to_sml(uts_observations, prior_observations, forecasting_step):
    """Splits a given UTS into multiple input rows where each input row has a specified number of timestamps and the output is a single timestamp.
    
    Parameters:
    uts_observations -- 1D np array (of UTS data to transform to SML data with size  b rows/length x 1 dimension)
    prior_observations -- py int (of all observations before we get to where we want to start making the predictions)
    forecasting_step -- py int (of how far out to forecast, 1 only the next timestamp, 2 the next two timestamps, ... n the next n timestamps)
    
    Return:
    agg.values -- np array (of new sml data)
    """
    
    df = pd.DataFrame(uts_observations)
    cols = list()
    
    lag_col_names = []
    count_lag = 0
    # input sequence (t-n, ... t-1)
    for prior_observation in range(prior_observations, 0, -1):
        # print("prior_observation: ", prior_observation)
        cols.append(df.shift(prior_observation))
        new_col_name = "t - " + str(prior_observation)
        # print(new_col_name)
        lag_col_names.append(new_col_name)
        
    
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, forecasting_step):
        cols.append(df.shift(-i))
        # print(f"t + {i}")
        if i == 0:
            new_col_name = f"t"
        else:
            new_col_name = f"t + {i}"
        # print(new_col_name)
        lag_col_names.append(new_col_name)
        
        # put it all together
        uts_sml_df = pd.concat(cols, axis=1) 
        uts_sml_df.columns=[lag_col_names]
        # drop rows with NaN values
        uts_sml_df.dropna(inplace=True)
    
    # print(uts_sml_df)
    
    # colums to use to make prediction for last col
    X_train = uts_sml_df.iloc[:, 0: -1]
    
    # last column
    y_train = uts_sml_df.iloc[:, [-1]]
    return uts_sml_df

In [ ]:
n_steps = 3
output_size = 3
converted_seq_df = convert_uts_sequence_to_sml(observations, n_steps, output_size)

In [ ]:
converted_seq_df

,t - 3,t - 2,t - 1,t,t + 1,t + 2
3,10.0,20.0,30.0,40,50.0,60.0
4,20.0,30.0,40.0,50,60.0,70.0
5,30.0,40.0,50.0,60,70.0,80.0
6,40.0,50.0,60.0,70,80.0,90.0


In [ ]:
forecast_X_train_df = converted_seq_df.iloc[:, :n_steps]
forecast_X_train_df

,t - 3,t - 2,t - 1
3,10.0,20.0,30.0
4,20.0,30.0,40.0
5,30.0,40.0,50.0
6,40.0,50.0,60.0


In [ ]:
forecast_y_train_df = converted_seq_df.iloc[:, -output_size:]
forecast_y_train_df

,t,t + 1,t + 2
3,40,50.0,60.0
4,50,60.0,70.0
5,60,70.0,80.0
6,70,80.0,90.0


In [ ]:
forecast_X_test_df = converted_seq_df.iloc[[-1], -n_steps:]
forecast_X_test_df

,t,t + 1,t + 2
6,70,80.0,90.0


In [ ]:
predict_X_train_df = converted_seq_df.iloc[:-1, :n_steps]
predict_X_train_df

,t - 3,t - 2,t - 1
3,10.0,20.0,30.0
4,20.0,30.0,40.0
5,30.0,40.0,50.0


In [ ]:
predict_y_train_df = converted_seq_df.iloc[:-1, -output_size:]
predict_y_train_df

,t,t + 1,t + 2
3,40,50.0,60.0
4,50,60.0,70.0
5,60,70.0,80.0


In [ ]:
predict_X_test_df = converted_seq_df.iloc[[-n_steps], -n_steps:]
predict_X_test_df

,t,t + 1,t + 2
4,50,60.0,70.0


In [ ]:
predict_y_test_df = converted_seq_df.iloc[[-1], -output_size:]
predict_y_test_df

,t,t + 1,t + 2
6,70,80.0,90.0


# Book's implementation
- Keras

## Forecast model

In [ ]:
from keras.models import Sequential 
from keras.layers import Dense

forecast_model = Sequential() 
forecast_model.add(Dense(100, activation='relu' , input_dim=n_steps)) 
forecast_model.add(Dense(output_size)) 
forecast_model.compile(optimizer='adam' , loss='mse') 

predict_model = Sequential() 
predict_model.add(Dense(100, activation='relu' , input_dim=n_steps)) 
predict_model.add(Dense(output_size)) 
predict_model.compile(optimizer='adam' , loss='mse') 

In [ ]:
# fit model 
forecast_model.fit(forecast_X_train_df, forecast_y_train_df, epochs=2000, verbose=0)

In [ ]:
forecast_X_test = np.array(forecast_X_test_df)
forecast_X_test, forecast_X_test.shape[0]

In [ ]:
X_test = forecast_X_test.reshape((forecast_X_test.shape[0]), n_steps)
X_test

In [ ]:
forecasts = forecast_model.predict(X_test, verbose=0)
forecasts

## Predict model

In [ ]:
predict_model.fit(predict_X_train_df, predict_y_train_df, epochs=2000, verbose=0)

In [ ]:
predict_X_test = np.array(predict_X_test_df)
predict_X_test, predict_X_test.shape[0]

In [ ]:
predict_X_test = forecast_X_test.reshape((predict_X_test.shape[0]), n_steps)
predict_X_test

In [ ]:
book_model_predictions = predict_model.predict(predict_X_test, verbose=0)
book_model_predictions

In [ ]:
predict_y_test_df

In [ ]:
EvaluationMetric.eval_mse(predict_y_test_df, book_model_predictions, False)

# My implementation
- PyTorch using my library

## Forecast model

In [ ]:
hidden_size = 100

mlp_forecast_model = MLP(n_steps, hidden_size, output_size)
mlp_forecast_model

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(mlp_forecast_model.parameters())
N_epochs = 2000
configs = [criterion, optimizer, N_epochs]

train_forecast_mlp_model = mlp_forecast_model.train(forecast_X_train_df, forecast_y_train_df, configs)

In [ ]:
X_test_df = forecast_X_test_df.iloc[[-1], -n_steps:].copy()
X_test_df

In [ ]:
X_test = torch.tensor(X_test_df.values, dtype=torch.float32)
X_test

In [ ]:
mlp_forecasts = mlp_forecast_model.predict(X_test, n_steps)
mlp_forecasts

# Prediction model

In [ ]:
hidden_size = 100

mlp_predict_model = MLP(n_steps, hidden_size, output_size)
mlp_predict_model

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(mlp_predict_model.parameters())
N_epochs = 2000
configs = [criterion, optimizer, N_epochs]

train_predict_mlp_model = mlp_predict_model.train(predict_X_train_df, predict_y_train_df, configs)

In [ ]:
X_test_df = predict_X_test_df.iloc[[-1], -n_steps:].copy()
X_test_df

In [ ]:
X_test = torch.tensor(X_test_df.values, dtype=torch.float32)
X_test

In [ ]:
mlp_predictions = mlp_predict_model.predict(X_test, n_steps)
mlp_predictions

In [ ]:
predict_y_test_df

In [ ]:
EvaluationMetric.eval_mse(predict_y_test_df, mlp_predictions, False)